In [ ]:
import math

def dobot_arm_angle(x: float, y: float, z: float):
    """
    Dobot Magicianのアーム角度を計算する関数

    パラメータ:
        x, y, z: エンドエフェクタの位置座標

    戻り値:
        (成功, A1, A2, A3, J1, J2, J3)
        成功がTrueの場合、A1, A2, A3は主軸回転角度、アーム1・アーム2の角度（度単位）
        J1, J2, J3はDobot表示用に変換された角度
        成功がFalseの場合、与えた位置は到達不可能と判断されたことを意味する
    """
    # 定数（各長さはDobor Magician固有の寸法）
    L1 = 0.0    # アーム基点の高さ（Dobor Magicianでは0）
    L2 = 135.0  # アーム1の辺の長さ
    L3 = 147.0  # アーム2の辺の長さ
    L4 = 60.0   # エンドエフェクタの突き出し部分（常に水平）

    # エンドエフェクタ位置からX-Y平面上の距離
    distance_xy = math.sqrt(x * x + y * y)
    # エンドエフェクタの突き出し部分を差し引いた、実際のアーム部分の長さ
    l = distance_xy - L4

    # X-Y平面上のアーム長が十分でなければ計算不能
    if l <= L4:
        return False, None, None, None, None, None, None

    # アーム基点からエンドエフェクタ先端までの直線距離
    ld = math.sqrt(l**2 + (z - L1)**2)

    # ldが正でかつ、アーム1とアーム2の長さの合計未満である必要がある
    if ld <= 0 or ld >= (L2 + L3):
        return False, None, None, None, None, None, None

    # 主軸の回転角度（X-Y平面上の角度）
    th1 = math.atan2(y, x)

    # 基点から先端までの仰角（水平からの角度）
    phi = math.atan2(z - L1, l)

    # アーム1の角度計算：余弦定理を利用
    cos_val = (ld**2 + L2**2 - L3**2) / (2.0 * ld * L2)
    # 数値誤差対策で[-1, 1]にクランプ
    cos_val = max(min(cos_val, 1.0), -1.0)
    th2 = phi + math.acos(cos_val)

    # アーム2のアーム1からの角度：arcsinを利用
    sin_val = (ld**2 - L2**2 - L3**2) / (2.0 * L2 * L3)
    sin_val = max(min(sin_val, 1.0), -1.0)
    th3 = math.asin(sin_val) + math.pi / 2.0

    # ラジアンから度へ変換
    A1 = math.degrees(th1)
    A2 = math.degrees(th2)
    A3 = math.degrees(th3)

    # Dobotの表示用角度に変換
    J1 = A1
    J2 = 90.0 - A2
    J3 = 180.0 - A2 - A3

    return True, A1, A2, A3, J1, J2, J3

success, A1, A2, A3, J1, J2, J3 = dobot_arm_angle(259, 0, -8.5 )
if success:
    print("計算成功:")
    print(f"A1: {A1:.2f}°, A2: {A2:.2f}°, A3: {A3:.2f}°")
    print(f"J1: {J1:.2f}°, J2: {J2:.2f}°, J3: {J3:.2f}°")
else:
    print("到達不可能な位置です。")


計算成功:
A1: 0.00°, A2: 22.42°, A3: 133.50°
J1: 0.00°, J2: 67.58°, J3: 24.08°
